# Exploratory Analysis for Starbucks Data and Preprocessing

In [199]:
import os

In [200]:
os.listdir()

['.ipynb_checkpoints',
 'Initial Analysis and Preprocessing.ipynb',
 'offers_transformed.csv',
 'Original Datasources (Do Not Modify)',
 'portfolio.csv',
 'portfolio_transformed.csv',
 'profile.csv',
 'profile_transformed.csv',
 'transactions_transformed',
 'transcript.csv']

Please verify that you've unzipped the file archive & that portfolio.csv, profile.csv, and transcript.csv are all present in your current directory.

## Loading data

In [201]:
import pandas as pd

In [202]:
port = pd.read_csv('portfolio.csv', index_col = 0) # index col = 0 since the index is unnamed in the csvs
prof = pd.read_csv('profile.csv', index_col = 0)
tran = pd.read_csv('transcript.csv', index_col = 0)

## Examining and Preprocessing Data

### Profile Data

In [203]:
prof.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [204]:
prof.isna().sum() # Looks like we are missing the same amount of gender / income.

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

In [205]:
(prof['age'] == 118).value_counts()

False    14825
True      2175
Name: age, dtype: int64

In [206]:
print('Missing data makes up about {}% of total profile rows'.format(round(2175/(14825+2175), 3) * 100))

Missing data makes up about 12.8% of total profile rows


So, all of the missing values occur on the same rows - where one column is missing data, so are the others (gender, age, income). For now, lets just drop them.

In [207]:
prof.dropna(inplace = True)

In [208]:
prof.isna().any() # Verify there are no missing values in prof

gender              False
age                 False
id                  False
became_member_on    False
income              False
dtype: bool

### Transcript Data

In [209]:
tran.isna().sum() # No missing values for tran file!

person    0
event     0
value     0
time      0
dtype: int64

In [210]:
tran.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [211]:
tran['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

### Writing Functions for Parsing the "Value Column"
The value column contains different information about value (offer id, reward, transaction amount) depending on what kind of "event" row it is - so we write a function to parse this information into their own columns and then iterate through the dataframe.

In [212]:
import ast

In [213]:
tran['offer_id'] = None
tran['amount'] = None
tran['reward'] = None


def parse_event_value():
    if i[1]['event'] == 'offer received':
        tran.at[i[0], 'offer_id'] = ast.literal_eval(i[1]['value'])['offer id']
    elif i[1]['event'] == 'offer viewed':
        tran.at[i[0], 'offer_id'] = ast.literal_eval(i[1]['value'])['offer id']
    elif i[1]['event'] == 'offer completed':
        tran.at[i[0], 'reward'] = ast.literal_eval(i[1]['value'])['reward']
        tran.at[i[0], 'offer_id'] = ast.literal_eval(i[1]['value'])['offer_id']
    elif i[1]['event'] == 'transaction':
        tran.at[i[0], 'amount'] = ast.literal_eval(i[1]['value'])['amount']

In [214]:
for i in tran.iterrows():
    parse_event_value()

### Separation and Self-Joins
We separate out the transcript dataframe by 'event' to make joins easier, then rename the columns of each column BEFORE the join for clarity.

In [215]:
transactions_only = tran[tran['event'] == 'transaction'].copy().drop(['value','offer_id', 'reward'], axis = 1)
completed_only = tran[tran['event'] == 'offer completed'].copy().drop(['value', 'amount'], axis = 1)
received_only = tran[tran['event'] == 'offer received'].copy().drop(['value', 'amount', 'reward'], axis = 1)
viewed_only = tran[tran['event'] == 'offer viewed'].copy().drop(['value', 'amount', 'reward'], axis = 1)

In [216]:
transactions_only.columns

Index(['person', 'event', 'time', 'amount'], dtype='object')

In [217]:
transactions_only = transactions_only.rename(columns = {'event':'transaction', 'time':'transaction time'})

In [218]:
completed_only.columns

Index(['person', 'event', 'time', 'offer_id', 'reward'], dtype='object')

In [219]:
completed_only = completed_only.rename(columns = {'event':'completed', 'time':'completed time'})

In [220]:
received_only.columns

Index(['person', 'event', 'time', 'offer_id'], dtype='object')

In [221]:
received_only = received_only.rename(columns = {'event': 'received', 'time':'received time'})

In [222]:
viewed_only.columns

Index(['person', 'event', 'time', 'offer_id'], dtype='object')

In [223]:
viewed_only = viewed_only.rename(columns = {'event': 'viewed', 'time':'viewed time'})

Now, we rejoin the transaction data frame together using left joins to create a dataframe with one row for each transaction, and information about whether or not it was related to an offer. This dataframe is interesting for analysis, but isn't as useful for our predictions.

In [224]:
tran_updated = pd.merge(transactions_only, completed_only, how = 'left', \
                        left_on = ['person', 'transaction time'], right_on = ['person', 'completed time'])

In [225]:
tran_updated = pd.merge(tran_updated, received_only, how = 'left', \
                        left_on = ['person', 'offer_id'], right_on = ['person', 'offer_id'])

In [226]:
tran_updated = pd.merge(tran_updated, viewed_only, how = 'left', \
                        left_on = ['person', 'offer_id'], right_on = ['person', 'offer_id'])

In [227]:
tran_updated

,person,transaction,transaction time,amount,completed,completed time,offer_id,reward,received,received time,viewed,viewed time
0,02c083884c7d45b39cc68e1314fec56c,transaction,0,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0,34.56,offer completed,0.0,2906b810c7d4411798c6938adc9daaa5,2,offer received,0.0,offer viewed,0.0
2,54890f68699049c2a04d415abc25e717,transaction,0,13.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b2f1cd155b864803ad8334cdf13c4bd2,transaction,0,19.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,fe97aa22dd3e48c8b143116a8403dd52,transaction,0,18.97,offer completed,0.0,fafdcd668e3743c1bb461111dcafc2a4,2,offer received,0.0,offer viewed,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
175728,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175729,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175730,a00058cf10334a308c68e7631c529907,transaction,714,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175731,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, having we rejoin the offer data  together using left joins to create a dataframe with one row for each unique offer for each person, and information about whether or not it was 'turned into' or 'converted' into a transaction. This dataframe will make up the basis of our training data once joined with other demographic / offer information.

In [228]:
offer_updated = pd.merge(received_only,viewed_only, how = 'left', \
                         left_on = ['person', 'offer_id'], right_on = ['person', 'offer_id'])

In [229]:
offer_updated = pd.merge(offer_updated, completed_only,  how = 'left', 
                         left_on = ['person', 'offer_id'], right_on = ['person', 'offer_id'])

In [230]:
offer_updated = pd.merge(offer_updated, transactions_only, how = 'left', \
                         left_on = ['person', 'completed time'], right_on = ['person', 'transaction time'])

In [231]:
offer_updated

,person,received,received time,offer_id,viewed,viewed time,completed,completed time,reward,transaction,transaction time,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,offer viewed,6.0,offer completed,132.0,5,transaction,132.0,19.89
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,6.0,NaN,NaN,NaN,NaN,NaN,NaN
2,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,offer viewed,624.0,NaN,NaN,NaN,NaN,NaN,NaN
3,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,offer viewed,18.0,NaN,NaN,NaN,NaN,NaN,NaN
4,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,offer viewed,12.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
115604,d087c473b4d247ccb0abfef59ba12b0e,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,offer viewed,672.0,offer completed,636.0,10,transaction,636.0,20.78
115605,cb23b66c56f64b109d673d5e56574529,offer received,576,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,offer completed,156.0,2,transaction,156.0,21.62
115606,6d5f3a774f3d4714ab0c092238f3a1d7,offer received,576,2298d6c36e964ae4a3e7e9706d1fb8c2,offer viewed,588.0,NaN,NaN,NaN,NaN,NaN,NaN
115607,9dc1421481194dcd9400aec7c9ae6366,offer received,576,ae264e3637204a6fb9bb56bc8210ddfd,offer viewed,624.0,offer completed,594.0,10,transaction,594.0,12.57


It's rather ugly to have a value for "viewed" if the offer was viewed and NaN if it isn't, so we write a quick looping function to convert these columns to 0/1 (where 1 is TRUE and 0 is FALSE)

In [232]:
import numpy as np

In [233]:
transform_list = ['received', 'viewed', 'completed', 'transaction']

In [234]:
def encode_cols(df):
    for i in transform_list:
        try:
            df[i] = np.where(df[i].isnull(), 0, 1)
        except:
            pass

In [235]:
encode_cols(offer_updated)

In [236]:
offer_updated

,person,received,received time,offer_id,viewed,viewed time,completed,completed time,reward,transaction,transaction time,amount
0,78afa995795e4d85b5d9ceeca43f5fef,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,6.0,1,132.0,5,1,132.0,19.89
1,a03223e636434f42ac4c3df47e8bac43,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,6.0,0,NaN,NaN,0,NaN,NaN
2,a03223e636434f42ac4c3df47e8bac43,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,624.0,0,NaN,NaN,0,NaN,NaN
3,e2127556f4f64592b11af22de27a7932,1,0,2906b810c7d4411798c6938adc9daaa5,1,18.0,0,NaN,NaN,0,NaN,NaN
4,8ec6ce2a7e7949b1bf142def7d0e0586,1,0,fafdcd668e3743c1bb461111dcafc2a4,1,12.0,0,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
115604,d087c473b4d247ccb0abfef59ba12b0e,1,576,ae264e3637204a6fb9bb56bc8210ddfd,1,672.0,1,636.0,10,1,636.0,20.78
115605,cb23b66c56f64b109d673d5e56574529,1,576,2906b810c7d4411798c6938adc9daaa5,0,NaN,1,156.0,2,1,156.0,21.62
115606,6d5f3a774f3d4714ab0c092238f3a1d7,1,576,2298d6c36e964ae4a3e7e9706d1fb8c2,1,588.0,0,NaN,NaN,0,NaN,NaN
115607,9dc1421481194dcd9400aec7c9ae6366,1,576,ae264e3637204a6fb9bb56bc8210ddfd,1,624.0,1,594.0,10,1,594.0,12.57


In [237]:
encode_cols(tran_updated)

In [238]:
tran_updated

,person,transaction,transaction time,amount,completed,completed time,offer_id,reward,received,received time,viewed,viewed time
0,02c083884c7d45b39cc68e1314fec56c,1,0,0.83,0,NaN,NaN,NaN,0,NaN,0,NaN
1,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,1,0,34.56,1,0.0,2906b810c7d4411798c6938adc9daaa5,2,1,0.0,1,0.0
2,54890f68699049c2a04d415abc25e717,1,0,13.23,0,NaN,NaN,NaN,0,NaN,0,NaN
3,b2f1cd155b864803ad8334cdf13c4bd2,1,0,19.51,0,NaN,NaN,NaN,0,NaN,0,NaN
4,fe97aa22dd3e48c8b143116a8403dd52,1,0,18.97,1,0.0,fafdcd668e3743c1bb461111dcafc2a4,2,1,0.0,1,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
175728,b3a1272bc9904337b331bf348c3e8c17,1,714,1.59,0,NaN,NaN,NaN,0,NaN,0,NaN
175729,68213b08d99a4ae1b0dcb72aebd9aa35,1,714,9.53,0,NaN,NaN,NaN,0,NaN,0,NaN
175730,a00058cf10334a308c68e7631c529907,1,714,3.61,0,NaN,NaN,NaN,0,NaN,0,NaN
175731,76ddbd6576844afe811f1a3c0fbb5bec,1,714,3.53,0,NaN,NaN,NaN,0,NaN,0,NaN


At this point, tran_updated and offer_updated are mostly complete, so we move onto the portfolio.csv.

## Portfolio Data / Offer Information

In [239]:
port

,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"['web', 'email', 'mobile', 'social']",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"['web', 'email', 'mobile', 'social']",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"['email', 'mobile', 'social']",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"['web', 'email', 'mobile', 'social']",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"['web', 'email', 'mobile']",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [240]:
port['email'] = 0
port['social'] = 0
port['mobile'] = 0
port['web'] = 0

def parse_channels_value():
    channels = ast.literal_eval(i[1]['channels'])
    if 'email' in channels:
        port.at[i[0], 'email'] = 1
    else:
        port.at[i[0], 'email'] = 0 
    if 'mobile' in channels:
        port.at[i[0], 'mobile'] = 1
    else:
        port.at[i[0], 'mobile'] = 0
    if 'web' in channels:
        port.at[i[0], 'web'] = 1
    else:
        port.at[i[0], 'web'] = 0
    if 'social' in channels:
        port.at[i[0], 'social'] = 1
    else:
        port.at[i[0], 'social'] = 0

In [241]:
for i in port.iterrows():
    parse_channels_value()

In [242]:
port.drop(['channels'], inplace = True, axis = 1)

In [243]:
port

,reward,difficulty,duration,offer_type,id,email,social,mobile,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,0,1,1
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,1,1
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,1,1,1,0
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,0,1,1


# Exporting Transformed Code

In [244]:
prof.to_csv('profile_transformed.csv')
port.to_csv('portfolio_transformed.csv')
offer_updated.to_csv('offers_transformed.csv')
tran_updated.to_csv('transactions_transformed.csv')